# make dict sic

In [1]:
from collections import defaultdict
import pickle
import os, re

In [2]:
# mlb
IN_DIR = 'data/amazon_hierarchy_2'
mlb_dirs = [os.path.join(IN_DIR,d) for d in os.listdir(IN_DIR) if d.startswith('mlb_cat')]
mlbs = [pickle.load(open(d,'rb')) for d in sorted(mlb_dirs)]

In [3]:
# create reverse dicts to go from class to index
class_to_index = []
for mlb in mlbs:
    class_to_index.append({cls:i for i,cls in enumerate(mlb.classes_)})

In [4]:
# parent_to_child
parent_to_child = []
for i,d in enumerate(class_to_index[:-1]):
    dd = {}
    for key,val in d.items():
        dd[val] = [v for k,v in class_to_index[i+1].items() if k.startswith(key)]
    parent_to_child.append(dd)

In [5]:
# child_to_siblings
child_to_siblings = [{}]
for i,d in enumerate(class_to_index[:-1]):
    dd = {}
    for key,val in d.items():
        siblings = [v for k,v in class_to_index[i+1].items() if k.startswith(key)]
        for sibling in siblings:
            dd[sibling]=siblings
    child_to_siblings.append(dd)
# create mask

In [ ]:
pickle.dump(child_to_siblings,open(os.path.join(IN_DIR,'child_to_siblings.pkl'),'wb'), pickle.HIGHEST_PROTOCOL)

In [ ]:
pickle.dump(parent_to_child,open(os.path.join(IN_DIR,'parent_to_child.pkl'),'wb'), pickle.HIGHEST_PROTOCOL)

# make dict amazon

In [6]:
from collections import defaultdict
import pickle
import os, re
import pandas as pd

In [7]:
# mlb
IN_DIR = 'data/amazon_hierarchy_2'
DF_DIR = 'data/amazon_hierarchy.pkl'
mlb_dirs = [os.path.join(IN_DIR,d) for d in os.listdir(IN_DIR) if d.startswith('mlb_cat')]
mlbs = [pickle.load(open(d,'rb')) for d in sorted(mlb_dirs)]

In [15]:
df = pd.read_pickle(DF_DIR)
df = df[df['train/test'].isin(['train','test'])]

In [16]:
df.head()

,categories,train/test,text,cat1,cat0,cat2,hierarchy
0000025240,"[books, new, used & rental textbooks]",train,alan a . block is emeritus professor of crime ...,[new],[books],[used & rental textbooks],[books > new > used & rental textbooks]
0001004743,"[books, children's books, classics]",train,grade 4-8 ? the law of the jungle ? intense co...,[children's books],[books],[classics],[books > children's books > classics]
0001042335,"[books, new, used & rental textbooks]",train,william shakespeare is widely regarded as the ...,[new],[books],[used & rental textbooks],[books > new > used & rental textbooks]
0001046349,"[books, literature & fiction, contemporary]",train,"`` no matter how appalling , malicious and pre...",[literature & fiction],[books],[contemporary],[books > literature & fiction > contemporary]
0001046438,"[books, literature & fiction, contemporary]",train,"fry is a british polymath -- actor , journalis...",[literature & fiction],[books],[contemporary],[books > literature & fiction > contemporary]


In [17]:
hierarchy = set()
for h in df.categories.values:
    hierarchy.add(' > '.join(h))
hierarchy = [h.split(' > ') for h in hierarchy]
len(hierarchy)

2487

In [18]:
# create reverse dicts to go from class to index
# mlb
mlb_dirs = [os.path.join(IN_DIR,d) for d in os.listdir(IN_DIR) if d.startswith('mlb_cat')]
mlbs = [pickle.load(open(d,'rb')) for d in sorted(mlb_dirs)]
class_to_index = []
for mlb in mlbs:
    class_to_index.append({cls:i for i,cls in enumerate(mlb.classes_)})

In [19]:
# parent_to_child
parent_to_child = [defaultdict(set) for _ in mlb_dirs[:-1]]
for hier in hierarchy:
    for i in range(len(hier)-1):
        key = class_to_index[i][hier[i]]
        val = class_to_index[i+1][hier[i+1]]
        parent_to_child[i][key].add(val)
parent_to_child = [{key:sorted(val) for key,val in p.items()} for p in parent_to_child]

In [20]:
# child_to_siblings
child_to_siblings = [{}]
for dd in parent_to_child:
    d = {}
    for val in dd.values():
        for v in val:
            d[v] = sorted(val)
    child_to_siblings.append(d)

In [21]:
pickle.dump(child_to_siblings,open(os.path.join(IN_DIR,'child_to_siblings.pkl'),'wb'), pickle.HIGHEST_PROTOCOL)
pickle.dump(parent_to_child,open(os.path.join(IN_DIR,'parent_to_child.pkl'),'wb'), pickle.HIGHEST_PROTOCOL)